## Motivation
Currently AI is advancing in the field of healthcare to improve detection of malignant tumors, give treatment recommendations, engage patients and support in administrative activities (Davenport and Kalakota 2019). Our goal is to contribute to this field by applying a neural network with transfer learning on a dataset with the aim to detect malignant cells of breast cancer. 

According to Krebsliga Schweiz (2021), there are 6’250 new cases and 1’410 deaths associated with breast cancer in Switzerland every year. Early diagnosis and treatment are a key to increasing the 5-year survival rate of patients.  

From a technical standpoint we want to investigate the performance differences between neural networks with and without transfer learning in the field of tumor detection.

## Data

We use the Kaggle dataset: Breast Histopathology Images, which contains 277’524 images that are classified whether the sample is positive or negative for Invasive Ductal Carcinoma (IDC). Therefore, we face a binary classification problem with this dataset. The sample dataset contains images scanned at 40x zoom that are prepared in 50 x 50-pixel patches.

[Kaggle Dataset](https://www.kaggle.com/paultimothymooney/breast-histopathology-images)

#### Import packages

In [1]:
import pandas as pd
import numpy as np
import datetime

# used to access folder structures
import os

# used to open images
import PIL

import tensorflow as tf
import keras




from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import train_test_split

# For Image Data Augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.callbacks import ReduceLROnPlateau

from tensorflow.keras.layers import Flatten, Dense, BatchNormalization, Activation, Dropout

from tensorflow.keras import layers

# Data preparation
* Initialize the path to all the images
* Create a dataframe with 3 collumns, which store the patient Id, the path to the picture and the picture itself
* Crete the new file strucure 

In [2]:
base_path = "images/IDC_regular_ps50_idx5/"
folder = os.listdir(base_path)
print("No. of Patients total:",len(folder))

total_images = 0
for n in range(len(folder)):
    patient_id = folder[n]
    for c in [0, 1]:
        patient_path = base_path + patient_id
        class_path = patient_path + '/' + str(c) + '/'
        subfiles = os.listdir(class_path)
        total_images += len(subfiles)
        
print("Total Images in dataset: ", total_images )

No. of Patients total: 279
Total Images in dataset:  277524


In total there are 279 patients in our dataset, from images of those patients in total 277'524 patches were created which each consist of a 50x50 pixel patch. While the number of patches to be analysed consists of a good number of samples to train a neural network, the relatively low number of patients indicates that further verification of our results would be needed to confirm generalizablility.  

Next we iterate over the

In [3]:
# create an empty dataframe with a column for each the patient id,
# the path to the image and the target label for each patch
data = pd.DataFrame(index=np.arange(0, total_images), columns=["patient_id", "path", "target"])

patientData = pd.DataFrame(index=np.arange(0, len(folder)), columns=["patient_id", "nrPos", "nrNeg"])

k = 0
n = 0
# Iterate over all patients (1 folder = 1 patient)
for n in range(len(folder)):
    
    # Fill the patient Data dataframe with the patient and the number of pos and neg patches
    if n > 0:
        patientData.iloc[n-1]["patient_id"] = patient_id
        patientData.iloc[n-1]["nrPos"] = nrPos
        patientData.iloc[n-1]["nrNeg"] = nrNeg
    
    nrPos = 0
    nrNeg = 0
    
    patient_id = folder[n]
    patient_path = base_path + patient_id 
    
    # Iterate over the two subfolders with the negative and positive patches 
    for c in [0,1]:
        
        # Count the number of positive and negative patches per patient
        if c == 0: nrNeg += 1
        else: nrPos += 1
        
        class_path = patient_path + "/" + str(c) + "/"
        subfiles = os.listdir(class_path)
        
        # Iterate over the images in the subfolder and fill the dataframe
        for m in range(len(subfiles)):
            image_path = subfiles[m]
            data.iloc[k]["path"] = class_path + image_path
            data.iloc[k]["target"] = c
            data.iloc[k]["patient_id"] = patient_id
            k += 1  

data.head()

IndexError: single positional indexer is out-of-bounds

In [ ]:
X_data=[]
y_data=[]
resized = 0

for index, row in data[:].iterrows():
    image = PIL.Image.open(row['path'])
    npImage = np.asarray(image)
    
    # Resize images with format different than our 50x50 patches
    if npImage.shape != (50, 50, 3):
        resized += 1
        image = image.resize((50, 50))
        npImage = np.asarray(image)
    X_data.append(npImage)
    y_data.append(row['target'])
    
    
print('X_data shape: ', np.array(X_data).shape)
print('y_data shape: ', np.array(y_data).shape)

print('In total %d patches had to be resized, since the format differed from 50x50'%resized)

In [ ]:
#Train-validation-test split

x_train,x_test,y_train,y_test=train_test_split(np.asarray(X_data),np.asarray(y_data),test_size=1/6)

x_train,x_val,y_train,y_val=train_test_split(x_train,y_train,test_size=.3)

#Dimension of the kaggle dataset
print((x_train.shape,y_train.shape))
print((x_val.shape,y_val.shape))
print((x_test.shape,y_test.shape))

input_shape=x_train.shape[1:]
input_shape

In [ ]:
#Image Data Augmentation

train_generator = ImageDataGenerator(rotation_range=20, horizontal_flip=True, vertical_flip=True)

val_generator = ImageDataGenerator(rotation_range=20, horizontal_flip=True, vertical_flip=True)

train_generator.fit(x_train)
val_generator.fit(x_val)

In [ ]:
#Learning Rate Annealer
lrr = ReduceLROnPlateau(monitor='val_accuracy',
                       factor=.1,
                       patience=3,
                       min_lr=1e-8,
                       verbose=2)


#Early stopping callback
es = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, verbose=2,
                                      mode='auto', baseline=None, restore_best_weights=True)

In [ ]:
name="5x5ConvX2-3x3ConvX2_512DenseWith0.2DropoutX2"
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        
        layers.Conv2D(64,kernel_size=(5,5),activation='relu'),
        layers.BatchNormalization(),
        layers.Conv2D(64,kernel_size=(5,5),activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=(2,2)),
        
        layers.Conv2D(64,kernel_size=(3,3),activation='relu'),
        layers.BatchNormalization(),
        layers.Conv2D(64,kernel_size=(3,3),activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=(2,2)),

        layers.Flatten(),
        layers.Dense(1024,activation='relu'),
        layers.Dropout(0.4),
        layers.Dense(1024,activation='relu'),
        layers.Dropout(0.4),

        layers.Dense(1, activation='sigmoid')    
    
    ],name=name
)

In [ ]:
model.summary()

In [ ]:
batch_size = 128
epochs = 50

model.compile(loss="binary_crossentropy", optimizer=Adam(epsilon=1e-07, learning_rate=0.001), metrics=["accuracy"])

history=model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1,callbacks=[es, lrr])

In [ ]:
from tensorflow import keras
from tensorflow.keras.regularizers import l2

input = tf.keras.Input(shape=input_shape)
beforeModel = tf.keras.layers.UpSampling2D()(input)
beforeModel = tf.keras.layers.UpSampling2D()(beforeModel)
#beforeModel = tf.keras.layers.UpSampling2D()(beforeModel)
print(beforeModel)
resnet = tf.keras.applications.ResNet50(include_top=False,weights='imagenet',input_shape=(200,200,3))
resnet.trainable=False

x = resnet(beforeModel,training=False)
x = keras.layers.Flatten()(x)

x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dense(1024,
                       kernel_regularizer=l2(0.001),
                       bias_regularizer=l2(0.001),
                       activation='relu')(x) # dense layer 1 
'''
x = keras.layers.Dropout(0.3)(x)

x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dense(1024,
                       activation='relu',
                       kernel_regularizer=l2(0.001),
                       bias_regularizer=l2(0.001))(x) # dense layer 2
x = keras.layers.Dropout(0.2)(x) 
'''

output = keras.layers.Dense(units=1, activation='sigmoid')(x)

In [ ]:
model = keras.Model(inputs = input, outputs = output)
model.summary()

In [ ]:
model.compile(loss="binary_crossentropy", optimizer=Adam(epsilon=0.1, learning_rate=0.001), metrics=["accuracy"])
log_dir= os.path.join('logs','ResNet50',datetime.datetime.now().strftime("%Y%m%d-%H%M%S"),'')

# tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

history=model.fit(train_generator.flow(x_train, y_train),
                  batch_size=batch_size, epochs=epochs,
                  validation_data=val_generator.flow(x_val, y_val),
                  callbacks=[es, lrr]) 